In [1]:
from pandas import DataFrame as df
import pandas as pd
import numpy as np
from toolz import thread_first,curry, thread_last

In [16]:
d1 = df(np.random.random([5,4]),columns=['b 1','c 1','d 1','e 1'])
d1['a'] = pd.Series(range(5))
d2 = df(np.random.random([5,1]),columns=['b2'])
d2['a'] = pd.Series(range(5))

In [17]:
m1 = pd.merge(d1,d2,on='a')

In [18]:
m1

,b 1,c 1,d 1,e 1,a,b2
0,0.581383,0.420767,0.873858,0.996229,0,0.296850
1,0.719407,0.556291,0.850606,0.984559,1,0.639522
2,0.088953,0.276311,0.020649,0.223271,2,0.741933
3,0.136980,0.537088,0.414217,0.254694,3,0.365032
4,0.539323,0.476245,0.842825,0.981016,4,0.158364


In [11]:
r = lambda n: np.random.random(n)
n = 10000
x = pd.DataFrame(dict(a = np.floor(r(n) * 10),
                      b = r(n),
                      c = r(n)))

In [12]:
def curry_funcs(funcs):
    for func in funcs:
        try: 
            exec('global {}; {} = curry({})'.format(*[func]*3))
        except:
            exec('{} = curry({})'.format(*[func]*2))

In [32]:
funcs = [df.mean,df.std,df.sem,df.count,df.min,df.max]
names = ['avg','std','sem','count','min','max']

@curry
def summarize(dataframe,funcs = [],names = []):
    summary = pd.concat([df(f(dataframe)).T for f in funcs])
    summary['Function'] = names
    return summary.reset_index(drop=True)

print summarize(x,funcs,names)

              a             b             c Function
0      4.511300      0.495966      0.494184      avg
1      2.864942      0.290479      0.288960      std
2      0.028649      0.002905      0.002890      sem
3  10000.000000  10000.000000  10000.000000    count
4      0.000000      0.000027      0.000281      min
5      9.000000      0.999989      0.999681      max


In [38]:
y = x.copy()
y['title'] = 'bob'
# print y.head()
print summarize(y,funcs,names)

            a             b             c  title Function
0      4.5113     0.4959657     0.4941839    NaN      avg
1    2.864942     0.2904786     0.2889601    NaN      std
2  0.02864942   0.002904786   0.002889601    NaN      sem
3       10000         10000         10000  10000    count
4           0  2.711458e-05  0.0002813152    bob      min
5           9     0.9999885     0.9996808    bob      max


In [33]:
def reset_index(dataframe):
    return dataframe.reset_index(drop=True)

snd = lambda x: x[1]

In [34]:
def summarize_groups(groups,funcs = [],names = []):
    return thread_last(groups,
                        (map, snd),
                        (map,summarize(funcs = funcs, 
                                       names = names)),
                        pd.concat,
                        reset_index)

In [39]:
summarize_groups(y.groupby('a'),funcs,names)

,a,b,c,title,Function
0,0,0.5009223,0.4783379,NaN,avg
1,0,0.2928977,0.2885636,NaN,std
2,0,0.009266873,0.009129747,NaN,sem
3,999,999,999,999,count
4,0,0.0006728686,0.000292597,bob,min
5,0,0.9990363,0.9959685,bob,max
6,1,0.5118278,0.4661265,NaN,avg
7,0,0.2897808,0.2851193,NaN,std
8,0,0.009406684,0.009255364,NaN,sem
9,949,949,949,949,count


In [35]:
x2 = pd.DataFrame(x.count()).T
x2['Function'] = 'mean'
x2[]


,a,b,c,Function
0,10000,10000,10000,mean


In [17]:
summarize(x,funcs,['s'])

[array([ 0.0288534 ,  0.00288482,  0.00287478]), array([ 2.88534014,  0.28848219,  0.28747837]), array([ 4.4965    ,  0.49559173,  0.50499939]), array([ 0.        ,  0.00022674,  0.00013953]), array([ 9.        ,  0.99989249,  0.99981598]), 10000]


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
def group_apply(group):
    

In [36]:
map(lambda x: x[1],gs)

IndexError: tuple index out of range

In [31]:
def add_constant_col(dataframe,name = "",val = ""):
    new_dataframe = dataframe.copy()
    new_dataframe[name] = val
    return new_dataframe

In [32]:
add_constant_col(x,'bob','santa')

,a,b,c,bob
0,0,0.780028,0.331236,santa
1,4,0.001477,0.177038,santa
2,7,0.553691,0.929238,santa
3,3,0.284791,0.451306,santa
4,3,0.963765,0.986582,santa
5,7,0.624703,0.737163,santa
6,3,0.028202,0.478833,santa
7,4,0.669904,0.314751,santa
8,7,0.058020,0.904912,santa
9,3,0.681391,0.656802,santa
